This workbook scrapes a foodies' ig page. 

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import datetime as dt
import requests

In [39]:
# Specify IG username to scrape
username = 'danielfooddiary'

# Specify location of chromedriver.exe
browser = webdriver.Chrome(executable_path=r'C:\Users\LENOVO\Downloads\chromedriver_win32\chromedriver.exe')

browser.get('https://www.instagram.com/'+username+'/?hl=en')

Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [40]:
# Put the user's IG pictures's links into a list called "links"
links = []
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
for link in body.findAll('a'):
     if re.match("/p", link.get('href')):
        links.append('https://www.instagram.com' + link.get('href'))
    
# print("Number of Instagram images: ", len(links))
# print(links)

In [47]:
# Put more pictures' links into "links" 

Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight/1.5);")
links = []
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
for link in body.findAll('a'):
     if re.match("/p", link.get('href')):
        links.append('https://www.instagram.com' + link.get('href'))

# Sleep time is required. If you don't use this IG may interrupt the script and doesn't scroll through pages

time.sleep(5) 

Pagelength = browser.execute_script("window.scrollTo(document.body.scrollHeight/1.5, document.body.scrollHeight/3.0);")
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
for link in body.findAll('a'):
     if re.match("/p", link.get('href')):
        links.append('https://www.instagram.com' + link.get('href'))
        
print("Number of Instagram images: ", len(links))
print(links)

Number of Instagram images:  48
['https://www.instagram.com/p/CCBS_tinKY7/', 'https://www.instagram.com/p/CB-wXfcH80_/', 'https://www.instagram.com/p/CB91xgsn3Uu/', 'https://www.instagram.com/p/CB7xBY1nRVL/', 'https://www.instagram.com/p/CB5mIPpH7mr/', 'https://www.instagram.com/p/CB3C1yUHazj/', 'https://www.instagram.com/p/CB2rLDMng_a/', 'https://www.instagram.com/p/CBxB0P-HFA1/', 'https://www.instagram.com/p/CBu3Uoon9GT/', 'https://www.instagram.com/p/CBuZvwfnfMQ/', 'https://www.instagram.com/p/CBsLvUsnGOG/', 'https://www.instagram.com/p/CBqK3-Tndkn/', 'https://www.instagram.com/p/CBmoNhznAxk/', 'https://www.instagram.com/p/CBlJEArn5Sk/', 'https://www.instagram.com/p/CBk6K0pHAdv/', 'https://www.instagram.com/p/CBiRXf0HyA_/', 'https://www.instagram.com/p/CBhtX11nKnh/', 'https://www.instagram.com/p/CBf5pDul7x0/', 'https://www.instagram.com/p/CBfVHZdDahq/', 'https://www.instagram.com/p/CBdPlnrH2IL/', 'https://www.instagram.com/p/CBZy5h0HdiS/', 'https://www.instagram.com/p/CBX5B4wnEFs/',

In [48]:
# Open the pictures in "links"
# Get details eg. timestamp, caption, comments, likes, etc. 
# Put details in dataframe

def get_date(created):
    return dt.datetime.fromtimestamp(int(created))

df = pd.DataFrame()
timestamp_list = []
caption_list = []
likes_list = [] 
comments_list = []
    
for link in links:
    req = requests.get(link).text
    soup = bs(req, "html.parser")
    scripts = soup.find_all("script")
    for script in scripts:
        if script.text[:18] == "window._sharedData":
            break

    data = json.loads(script.contents[0][21:-1])
    
    unix_time = data["entry_data"]["PostPage"][0]["graphql"]["shortcode_media"]["taken_at_timestamp"]
    human_time = get_date(unix_time)
    caption = str(data["entry_data"]["PostPage"][0]["graphql"]["shortcode_media"]["edge_media_to_caption"]["edges"][0]["node"]["text"])
    likes = str(data["entry_data"]["PostPage"][0]["graphql"]["shortcode_media"]["edge_media_preview_like"]["count"])
    
    timestamp_list.append(human_time)
    caption_list.append(caption)
    likes_list.append(likes)

    if "edge_media_preview_comment" in data["entry_data"]["PostPage"][0]["graphql"]["shortcode_media"]:
        comments = str(data["entry_data"]["PostPage"][0]["graphql"]["shortcode_media"]["edge_media_preview_comment"]["count"])
    
    comments_list.append(comments)
    
df["timestamp"] = timestamp_list
df["caption"] = caption_list
df["likes"] = likes_list
df["comments"] = comments_list

In [49]:
df.shape

(48, 4)

In [ ]:
df.to_excel("ig_foodies_posts.xlsx")  

Filter rows based on timestamp (before CB and after CB)   *tableau/jupyter

Using the caption column, do a wordcloud`  *tableau/jupyter

In [ ]:
tableau viz the other stuff 

In [ ]:
social network graph: nodes (12 fooedies & their commenters) & ties (comments)